In [13]:
import cv2
import os
import mediapipe as mp
import pandas as pd
import numpy as np
mp_hands = mp.solutions.hands

In [14]:
def get_hand_image(image):
    if(image is not None):
        width = image.shape[1]
        height = image.shape[0]
        with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = hands.process(image)
            image.flags.writeable = True
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = np.array([[lm.x * width, lm.y * height] for lm in hand_landmarks.landmark])
                    hand_bbox = cv2.boundingRect(landmarks.astype(np.float32))
                    # Check if the hand bounding box is valid
                    if hand_bbox[2] > 0 and hand_bbox[3] > 0:
                        # Extract hand image
                        hand_image = image[hand_bbox[1]:hand_bbox[1] + hand_bbox[3], hand_bbox[0]:hand_bbox[0] + hand_bbox[2]]
                        # Check if hand_image is not empty
                        if hand_image.size != 0:
                            # Resize hand image
                            hand_image = cv2.resize(hand_image, (64, 64))
                            return hand_image
                        else:
                            print("Error: Empty hand image")
                    else:
                        print("Error: Invalid hand bounding box")
            else:
                print("Error: No hand detected")
        # Return None if no hand image is found
        return None


In [15]:
def create_seq_of_frames(vidPath):
    sequence = []
    seq_len = 30
    
    vid = cv2.VideoCapture(vidPath)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    print("There are {} frames to be processed ".format(total_frames))
    buffer = np.zeros((64, 64,3))
    # Determine the step size to select frames evenly
    step = max(total_frames // seq_len, 1)
    for frame_num in range(0, total_frames, step):
        vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        _, frame = vid.read()

        hand_frame = get_hand_image(frame)
        # plt.imshow(hand_frame,cmap='gray')

        if hand_frame is None:
            sequence.append(buffer/255)
        else:
            sequence.append(hand_frame / 255)
            buffer = hand_frame
    # Apply pre-padding if necessary
    while len(sequence) < seq_len:
        sequence.append(buffer / 255)
    
    vid.release()
    return sequence[:seq_len] 

In [16]:
# create_seq_of_frames(r"C:\Users\Rommel\OneDrive\Pictures\Camera Roll\WIN_20240330_13_58_23_Pro.mp4")

In [17]:
# def create_seq_of_frames(vidPath):
#     sequence = []
#     vid = cv2.VideoCapture(vidPath)
#     total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("There are {} frames to be processed ".format(total_frames))
#     for frame_num in range(total_frames):
#         if(frame_num % 5 != 0):
#             continue
#         vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
#         _ , frame = vid.read()
#         frame.resize(64,64,3)
#         print("Processing frame {}".format(frame_num))
#         sequence.append(frame/255)
#         # cv2.imwrite('{}/{}.png'.format(vidPath[5:-4],frame_num), frame)
#         if(frame_num > 45):
#             break
#     vid.release()
#     return sequence

In [18]:
# def create_seq_of_frames(vidPath):
#     sequence = []
#     seq_len = 30
#     vid = cv2.VideoCapture(vidPath)
#     total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("There are {} frames to be processed ".format(total_frames))
    
#     # Determine the step size to select frames evenly
#     step = max(total_frames // seq_len, 1)
    
#     for frame_num in range(0, total_frames, step):
#         vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
#         _, frame = vid.read()
#         frame = cv2.resize(frame, (64, 64))
#         # print("Processing frame {}".format(frame_num))
#         sequence.append(frame / 255)
    
#     # Apply pre-padding if necessary
#     while len(sequence) < seq_len:
#         sequence.insert(0, np.zeros((64, 64, 3)))
    
#     vid.release()
#     return sequence[:seq_len]  # Ensure the returned sequence has a maximum length of 12 frames


In [19]:
def create_dataset(parentFolder,dataset):
    for file in os.listdir(parentFolder):
        filePath = "{}/{}".format(parentFolder,file)
        print("Processing " + filePath)
        dataset.append(create_seq_of_frames(filePath))
    return dataset

In [20]:
x = []
y = []

In [21]:
x = create_dataset(r"D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct",x)
x = create_dataset(r"D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\incorrect",x)

Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/20240330_180851.mp4
There are 49 frames to be processed 


Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/20240330_180908.mp4
There are 44 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/20240330_180911.mp4
There are 48 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/20240330_180914.mp4
There are 40 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/20240330_180923.mp4
There are 46 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/VID20240330225840.mp4
There are 57 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\correct/VID20240330225910.mp4
There are 23 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\incorrect/20240330_180854.mp4
There are 63 frames to be processed 
Processing D:\Projects\Codeshastra_ByteMe\Bodhi\Testing data\incorrect/20240330_180901.mp4
There are 38 frames t

In [ ]:
y = ['correct'] * 7 + ['incorrect'] * 7

In [ ]:
import pickle

In [ ]:
dataset = {
    "features":x,
    "labels":y
}

In [ ]:
with open('test_dataset.pkl','wb') as f:
    pickle.dump(dataset,f)